## AlexNet

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

### CIFAR10 데이터세트 불러오기
transforms.normalize은 특정 평균과 표준편차를 따르는 정규분포를 통해 이미지를 표준화하는 방법이다. CIFAR10은 3채널 컬러 이미지이므로 각 장의 평균과 표준편차를 정한다. 첫번째 0.5,0.5,0.5는 각 채널당 평균을 할당한 것이고 튜플로 입력한다. 두번째는 각 채널의 표준편차다. 평균과 표준편차는 학습전 가지고 있는 이미지로부터 계산하지만 이번에는 임의의 값 0.5를 기입한다.

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True) 

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
# CPU/GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'{device} is available.')

cpu is available.


AlexNet 구축하기
본래 AlexNet는 ImageNet 데이터를 위해 만들어졌다. Imagenet은 1000개의 클래스로분류되어 있는 256x256 또는 224x224 크기를 갖는 이미지이다. 따라서 크기가 32x32인 CIFAR10 이미지는 제대로 작동하지 않을 수 있다. 따라서 데이터에 맞게 필터와 크기와 보폭수를 조정해 모델을 구축한다

In [7]:
# Conv→ReLU→MaxPool→ Conv→ReLU→MaxPool→Conv→ReLU→ Conv→ReLU→Conv→ReLU→MaxPool→FC1→ReLU→FC2→ReLU→FC3

class AlexNet(nn.Module): # 클래스 생성
    def __init__(self): # 함수 구조 작성
        super(AlexNet, self).__init__()
        self.features = nn.Sequential( 
                        nn.Conv2d(3, 64, 3), nn.ReLU(), #CIFAR110은 RGB컬러 이미지다 따라서 입력 채널의 수가 3이므로 채널 수르 3으로 설정
                        nn.MaxPool2d(2, 2),
                        nn.Conv2d(64, 192, 3, padding=1), nn.ReLU(),
                        nn.MaxPool2d(2, 2),
                        nn.Conv2d(192, 384, 3, padding=1), nn.ReLU(),
                        nn.Conv2d(384, 256, 3, padding=1), nn.ReLU(),
                        nn.Conv2d(256, 256, 1), nn.ReLU(),
                        nn.MaxPool2d(2, 2)        
                        )
        # sequential을 사용하면 순차적으로 행해지는 연산을 한 번에 묶을 수 있다. nn.Sequential의 괄호 안은 작성 순서대로 연산이 수행된다.
        self.classifier = nn.Sequential(
                        nn.Dropout(0.5),
                        nn.Linear(256*3*3, 1024), nn.ReLU(),
                        nn.Dropout(0.5),
                        nn.Linear(1024, 512), nn.ReLU(),
                        nn.Linear(512, 10)
                        )    
        #Fully-connected layer로 구성된 self.classifer을 정의한다.
        
    def forward(self, x):
        x = self.features(x)
        x = x.view(-1, 256*3*3)
        x = self.classifier(x)    
        return x

### 손실 함수 및 최적화 방법 정의하기
다중 분류 문제에서는 Cross Entropy Loss를 기본으로 사용한다. 파이토치에서 제공하는 Cross Entropy Loss는 softmax계산까지 포함되어 있으므로 모델의 마지막 출력값에 별도의 softmax를 적용하지 않아도 된다. 그리고 GPU연산을 위해 모델을 불러 올대 .to(device)를 반드시 붙여준다.

In [8]:
criterion = nn.CrossEntropyLoss() # CrossEntropyLoss는 softmax 계산까지 포함되어 있으므로 모델의 마지막 output node에 별도의 활성화 함수를 사용하지 않아도 된다.
alexnet = AlexNet().to(device) # 모델 선언
optimizer = optim.Adam(alexnet.parameters(), lr=1e-3)

In [ ]:
# 모델의 학습 과정인 인공 신경망과 동일하다.
loss_ = [] # 그래프를 그리기 위한 loss 저장용 리스트 
n = len(trainloader) # 배치 개수

for epoch in range(50):  # 10번 학습을 진행한다.

    running_loss = 0.0
    for data in trainloader:

        inputs, labels = data[0].to(device), data[1].to(device) # 배치 데이터 
        
        optimizer.zero_grad()
        outputs = alexnet(inputs) # 예측값 산출 
        loss = criterion(outputs, labels) # 손실함수 계산
        loss.backward() # 손실함수 기준으로 역전파 선언
        optimizer.step() # 가중치 최적화

        # print statistics
        running_loss += loss.item()

    loss_.append(running_loss / n)    
    print('[%d] loss: %.3f' %(epoch + 1, running_loss / len(trainloader)))

[1] loss: 1.589
[2] loss: 1.172
[3] loss: 0.996
[4] loss: 0.884
[5] loss: 0.801
[6] loss: 0.734
[7] loss: 0.683
[8] loss: 0.635


In [ ]:
#학습 손실 함수 그래프 그리기
plt.plot(loss_)
plt.title("Training Loss")
plt.xlabel("epoch")
plt.show()

In [ ]:
#파이토치 모델 저장 및 불러오기
print(alexnet)
PATH = './models/cifar_alexnet.pth' # 모델 저장 경로 
torch.save(alexnet.state_dict(), PATH) # 모델 저장

In [ ]:
# 모델 불러오기는 엄밀히 말하자면 모델의 파라메타를 불러오는 것이다. 따라서 모델의 뼈대를 먼저 선언하고
# 모델의 파라메타를 불러와 pretrained model을 만든다.

alexnet = AlexNet().to(device) # 모델 선언
alexnet.load_state_dict(torch.load(PATH)) # 모델 파라메타 불러오기

### 모델 정확도 구하기 / 평가

In [ ]:
# 평가 데이터를 이용해 정확도를 구해보자.
# output은 미니배치의 결과가 산출되기 때문에 for문을 통해서 test 전체의 예측값을 구한다.

correct = 0
total = 0
with torch.no_grad():
    alexnet.eval()
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = alexnet(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0) # 개수 누적(총 개수)
        correct += (predicted == labels).sum().item() # 누적(맞으면 1, 틀리면 0으로 합산)
        
print('Test accuracy: %.2f %%' % (100 * correct / total))

### ResNet
ImageNet 데이터에 맞춰진 ResNet은 기본층에서 7x7 필터를 사용하는 합성곱과 3x3 맥스 풀링을 사용한다. CIFAR10은 이미지 사이즈가 ImageNet 이미지보다 훨씬 작기 때문에 그림과 같이 기본층의 합성곱 필터 사이즈를 3x3으로 줄이고 맥스 풀링을 생략한다. 또한 Resnet18과 ResNet34는 동일한 블록을 사용하기 때문에 층을 조절하여 두 모델을 구현할 수 있다.

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

In [ ]:
class ResidualBlock(nn.Module):

    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.stride = stride
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.conv_block = nn.Sequential(
            nn.Conv2d(self.in_channels, self.out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(self.out_channels),
            nn.ReLU(),
            nn.Conv2d(self.out_channels, self.out_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(self.out_channels))
        #학습을 빠르게 할 수 있는 배치 정규화를 층 사이에 적용한다.
        #배치 정규화는 각 배치의 평규과 분산을 이용해서 데이터를 정규화하는 방법이다.
        if self.stride != 1 or self.in_channels != self.out_channels:
            self.downsample = nn.Sequential(
                            nn.Conv2d(self.in_channels, self.out_channels, kernel_size=1, stride=stride, bias=False),
                            nn.BatchNorm2d(self.out_channels))
    def forward(self, x):
        out = self.conv_block(x)
        if self.stride != 1 or self.in_channels != self.out_channels:
            x = self.downsample(x)

        out = F.relu(x + out)
        return out

In [ ]:
class ResNet(nn.Module):
    def __init__(self, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.base = nn.Sequential(
                        nn.Conv2d(3, 64, kernel_size=3,stride=1, padding=1, bias=False),
                        nn.BatchNorm2d(64),
                        nn.ReLU())
        self.layer1 = self._make_layer(64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(512, num_blocks[3], stride=2)
        self.gap = nn.AvgPool2d(4) # 4: 필터 사이즈
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, out_channels, num_blocks, stride):
        
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            block = ResidualBlock(self.in_channels, out_channels, stride)
            layers.append(block)
            self.in_channels = out_channels
    
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.base(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.gap(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

def modeltype(model):
    if model == 'resnet18':
        return ResNet([2, 2, 2, 2])

    elif model == 'resnet34':
        return ResNet([3, 4, 6, 3])

In [ ]:
resnet = modeltype('resnet18').to(device)
print(resnet)
PATH = './models/cifar_resnet.pth' # 모델 저장 경로 

파이토치에서는 손쉽게 이용할 수 있는 다양한 신경망을 제공한다.

In [ ]:
import torchvision.models as models
alexnet = models.alexnet().to(divice)
alexnet.classifier[6] = nn.Linear(4096,5)

In [ ]:
resnet18 = models.resnet18().to(device)
vgg16 = models.vgg16().to(device)